# Partly Sunny with a Chance of Hashtags

## Keras - 1 Attempt

## Attempt 1

In [ ]:

import random
import numpy as np
import tensorflow as tf
import pandas as pd
import json
import time
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.pipeline import Pipeline
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

# Load data
train_df = pd.read_csv('train.csv.zip')
test_df = pd.read_csv('test.csv.zip')
sub_example = pd.read_csv('sampleSubmission.csv.zip', nrows=1)
id_col = sub_example.columns[0]
target_columns = list(sub_example.columns[1:])

# Prepare X and y
# Drop rows/columns with all missing in train
train_df = train_df.dropna(axis=1, how='all')

y = train_df[target_columns].values
X = train_df.drop(columns=target_columns + [id_col], errors='ignore')
X = X.dropna(axis=1, how='all')

# Prepare test features and ids
X_test = test_df.drop(columns=target_columns + [id_col], errors='ignore')
test_ids = test_df[id_col].reset_index(drop=True)

# Feature engineering: detect numeric and categorical columns
numeric_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_cols = [c for c in X.columns if X[c].dtype=='object' and X[c].nunique() <= 50]

# Preprocessing pipelines
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])
cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])
preprocessor = ColumnTransformer([
    ('num', num_pipeline, numeric_cols),
    ('cat', cat_pipeline, categorical_cols)
])

# Fit and transform
X_proc = preprocessor.fit_transform(X)
X_test_proc = preprocessor.transform(X_test)

# Model architecture decision
d, m = X_proc.shape[1], len(target_columns)
n_samples = X_proc.shape[0]

model = Sequential()
if n_samples < 10000 or d < 100:
    # small dataset
    units1 = min(d * 2, 128)
    units2 = min(d, 64)
    model.add(Dense(units1, activation='relu', input_shape=(d,)))
    model.add(Dropout(0.3))
    model.add(Dense(units2, activation='relu'))
    model.add(Dropout(0.3))
else:
    # larger dataset (unused here but provided)
    layers = [min(d * i, 1024) for i in (2, 1, 0.5, 0.25)]
    layers = [u for u in layers if u >= 16]
    model.add(Dense(layers[0], activation='relu', input_shape=(d,)))
    for u in layers[1:]:
        model.add(Dense(u, activation='relu'))
        model.add(tf.keras.layers.BatchNormalization())
        model.add(Dropout(0.4))

# Output layer for multi-label classification
model.add(Dense(m, activation='sigmoid'))

# Compile model
model.compile(
    optimizer=Adam(),
    loss='binary_crossentropy',
    metrics=['accuracy', Precision(name='precision'), Recall(name='recall')]
)

# Callbacks
callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=1),
    ModelCheckpoint('best_model.h5', save_best_only=True, monitor='val_loss', verbose=1)
]

# Training
start_time = time.time()
history = model.fit(
    X_proc, y,
    validation_split=0.2,
    epochs=100,
    batch_size=128,
    callbacks=callbacks,
    verbose=2
)
duration = time.time() - start_time

# Save results
res = {
    'training_accuracy': history.history['accuracy'][-1],
    'training_loss': history.history['loss'][-1],
    'validation_accuracy': history.history['val_accuracy'][-1],
    'validation_loss': history.history['val_loss'][-1]
}
with open('results.json', 'w') as f:
    json.dump(res, f)

# Predictions & Submission
raw_preds = model.predict(X_test_proc)
final = (raw_preds > 0.5).astype(int)

import pandas as pd
submission = pd.DataFrame(final, columns=target_columns)
submission.insert(0, id_col, test_ids)
submission.to_csv('submission_result.csv', index=False)

2025-07-09 22:55:49.475977: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752101749.499882  934364 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752101749.507159  934364 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1752101749.533052  934364 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1752101749.533070  934364 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1752101749.533072  934364 computation_placer.cc:177] computation placer alr

Epoch 1/100


/home/exouser/.local/lib/python3.12/site-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-07-09 22:55:53.366731: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected



Epoch 1: val_loss improved from inf to 0.69315, saving model to best_model.h5


488/488 - 2s - 5ms/step - accuracy: 4.3300e-04 - loss: 0.6931 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_accuracy: 1.2829e-04 - val_loss: 0.6931 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/100

Epoch 2: val_loss did not improve from 0.69315
488/488 - 1s - 2ms/step - accuracy: 4.3300e-04 - loss: 0.6931 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_accuracy: 1.2829e-04 - val_loss: 0.6931 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 3/100

Epoch 3: val_loss did not improve from 0.69315
488/488 - 1s - 2ms/step - accuracy: 4.3300e-04 - loss: 0.6931 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_accuracy: 1.2829e-04 - val_loss: 0.6931 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 4/100

Epoch 4: val_loss did not improve from 0.69315
488/488 - 1s - 2ms/step - accuracy: 4.3300e-04 - loss: 0.6931 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_accuracy: 1.2829e-04 - val_loss: 0.6931 - val_precision: 0.0000e+00 - val_recall: 0.0000e

## Keras Tuner - 1 Attempt

## Attempt 1

In [ ]:
import random
import numpy as np
import tensorflow as tf
import pandas as pd
import json
import time
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.pipeline import Pipeline
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

# Load data
train_df = pd.read_csv('train.csv.zip')
test_df = pd.read_csv('test.csv.zip')
sub_example = pd.read_csv('sampleSubmission.csv.zip', nrows=1)
id_col = sub_example.columns[0]
target_columns = list(sub_example.columns[1:])

# Prepare X and y
# Drop rows/columns with all missing in train
train_df = train_df.dropna(axis=1, how='all')

y = train_df[target_columns].values
X = train_df.drop(columns=target_columns + [id_col], errors='ignore')
X = X.dropna(axis=1, how='all')

# Prepare test features and ids
X_test = test_df.drop(columns=target_columns + [id_col], errors='ignore')
test_ids = test_df[id_col].reset_index(drop=True)

# Feature engineering: detect numeric and categorical columns
numeric_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_cols = [c for c in X.columns if X[c].dtype=='object' and X[c].nunique() <= 50]

# Preprocessing pipelines
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])
cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])
preprocessor = ColumnTransformer([
    ('num', num_pipeline, numeric_cols),
    ('cat', cat_pipeline, categorical_cols)
])

# Fit and transform
X_proc = preprocessor.fit_transform(X)
X_test_proc = preprocessor.transform(X_test)

# Model architecture decision
d, m = X_proc.shape[1], len(target_columns)
n_samples = X_proc.shape[0]

import keras_tuner as kt
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Precision, Recall

# Define early stopping and checkpoint
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True)

# Input dimension
n_features = X_proc.shape[1]

# HyperModel
class MyHyperModel(kt.HyperModel):
    def build(self, hp):
        layers = hp.Int('layers', 2, 8)
        units = hp.Int('units', 64, 1024, 64)
        act = hp.Choice('activation', ['relu'])
        drop = hp.Float('dropout', 0.0, 0.5)
        opt = hp.Choice('optimizer', ['adam'])
        lr = hp.Float('learning_rate', 1e-5, 0.01, sampling='log')

        inputs = Input(shape=(n_features,))
        x = inputs
        for _ in range(layers):
            x = Dense(units, activation=act)(x)
            x = Dropout(drop)(x)
        x = Dense(m, activation='sigmoid')(x)  # Output layer for multi-label classification
        model = Model(inputs, x)
        model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy', Precision(name='precision'), Recall(name='recall')])
        return model

# Tuner
bs = 64  # batch size
ep = 50  # epochs

tuner = kt.BayesianOptimization(
    MyHyperModel(),
    objective='val_loss',
    max_trials=10,
    executions_per_trial=1,
    seed=42,
    overwrite=True,
    project_name='bayesian_tuner'
)

if y is not None:
    tuner.search(
        X_proc, y,
        validation_split=0.2,
        batch_size=bs, epochs=ep,
        callbacks=[early_stopping, checkpoint]
    )
else:
    tuner.search(
        X_proc, y,
        validation_split=0.2,
        batch_size=bs, epochs=ep,
        callbacks=[early_stopping, checkpoint]
    )

model = tuner.hypermodel.build(
    tuner.get_best_hyperparameters(1)[0]
)

start_time = time.time()  # Start timing


# Retrain model with original callbacks and data
if y is not None:
    history = model.fit(
        X_proc, y,
        validation_split=0.2,
        epochs=100, batch_size=bs,
        callbacks=[early_stopping, checkpoint],
        verbose=2
    )
else:
    history = model.fit(
        X_proc, y,
        validation_split=0.2,
        epochs=100, batch_size=bs,
        callbacks=[early_stopping, checkpoint],
        verbose=2
    )

duration = time.time() - start_time  # Calculate duration


# Save results
res = {
    'training_accuracy': history.history['accuracy'][-1],
    'training_loss': history.history['loss'][-1],
    'validation_accuracy': history.history['val_accuracy'][-1],
    'validation_loss': history.history['val_loss'][-1]
}
with open('results.json', 'w') as f:
    json.dump(res, f)

# Predictions & Submission
raw_preds = model.predict(X_test_proc)
final = (raw_preds > 0.5).astype(int)

import pandas as pd
submission = pd.DataFrame(final, columns=target_columns)
submission.insert(0, id_col, test_ids)
submission.to_csv('submission_result.csv', index=False)

Trial 10 Complete [00h 03m 06s]
val_loss: 0.3097052574157715

Best val_loss So Far: 0.3097052574157715
Total elapsed time: 00h 32m 10s
Epoch 1/100


975/975 - 15s - 15ms/step - accuracy: 0.3206 - loss: 0.5541 - precision: 0.8844 - recall: 0.1655 - val_accuracy: 0.3242 - val_loss: 0.4544 - val_precision: 0.8863 - val_recall: 0.1654
Epoch 2/100


975/975 - 5s - 5ms/step - accuracy: 0.3210 - loss: 0.4044 - precision: 0.8844 - recall: 0.1656 - val_accuracy: 0.3242 - val_loss: 0.3681 - val_precision: 0.8863 - val_recall: 0.1654
Epoch 3/100


975/975 - 5s - 5ms/step - accuracy: 0.3210 - loss: 0.3492 - precision: 0.8844 - recall: 0.1656 - val_accuracy: 0.3242 - val_loss: 0.3349 - val_precision: 0.8863 - val_recall: 0.1654
Epoch 4/100


975/975 - 5s - 5ms/step - accuracy: 0.3210 - loss: 0.3273 - precision: 0.8844 - recall: 0.1656 - val_accuracy: 0.3242 - val_loss: 0.3212 - val_precision: 0.8863 - val_recall: 0.1654
Epoch 5/100


...

975/975 - 5s - 5ms/step - accuracy: 0.3210 - loss: 0.3098 - precision: 0.8844 - recall: 0.1656 - val_accuracy: 0.3242 - val_loss: 0.3097 - val_precision: 0.8863 - val_recall: 0.1654
Epoch 22/100


975/975 - 5s - 5ms/step - accuracy: 0.3210 - loss: 0.3098 - precision: 0.8844 - recall: 0.1656 - val_accuracy: 0.3242 - val_loss: 0.3097 - val_precision: 0.8863 - val_recall: 0.1654
Epoch 23/100


975/975 - 5s - 5ms/step - accuracy: 0.3210 - loss: 0.3098 - precision: 0.8844 - recall: 0.1656 - val_accuracy: 0.3242 - val_loss: 0.3097 - val_precision: 0.8863 - val_recall: 0.1654
Epoch 24/100
975/975 - 5s - 5ms/step - accuracy: 0.3210 - loss: 0.3098 - precision: 0.8844 - recall: 0.1656 - val_accuracy: 0.3242 - val_loss: 0.3097 - val_precision: 0.8863 - val_recall: 0.1654
Epoch 25/100
975/975 - 5s - 5ms/step - accuracy: 0.3210 - loss: 0.3098 - precision: 0.8844 - recall: 0.1656 - val_accuracy: 0.3242 - val_loss: 0.3097 - val_precision: 0.8863 - val_recall: 0.1654
Epoch 26/100
975/975 - 5s - 5ms/step - accuracy: 0.3210 - loss: 0.3098 - precision: 0.8844 - recall: 0.1656 - val_accuracy: 0.3242 - val_loss: 0.3097 - val_precision: 0.8863 - val_recall: 0.1654
Epoch 27/100
975/975 - 5s - 5ms/step - accuracy: 0.3210 - loss: 0.3098 - precision: 0.8844 - recall: 0.1656 - val_accuracy: 0.3242 - val_loss: 0.3097 - val_precision: 0.8863 - val_recall: 0.1654
Epoch 28/100
975/975 - 5s - 5ms/step -

2025-07-17 01:00:23.888568: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_23', 4 bytes spill stores, 4 bytes spill loads



1318/1318 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step


In [ ]:
print(duration)


177.957296
